In [6]:
# Code to get all the revised LC points (from rerunning EITfluxsum.py with revised bakeout dates independently for
# years 2018, 2019, 2020, 2021, 2022) into the same results file

# also revise the EIT pinhole flag to only apply to images which have filter = 'Clear'

In [33]:
import EITfunc.EITlook as eit

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clrs
import matplotlib.cm as cmx
import matplotlib.colorbar as cb
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import gridspec

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

import os

from astropy import units as u
from astropy.constants import G
from astropy.io import ascii, fits
from astropy.table import Table
from astropy.timeseries import LombScargle
import astropy.time
from astropy.coordinates import HeliocentricMeanEcliptic, SkyCoord

import sunpy.coordinates

import scipy.optimize as sciop
from scipy.stats import mode, binned_statistic

import time
import copy

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
EITmetadata = Table.read("./EIT/EITresults/EIT_headerMetadata.txt",format="ascii.basic")
print(np.shape(EITmetadata))
print(EITmetadata)
print(EITmetadata.colnames)

EITmetadata = EITmetadata[np.argsort(EITmetadata['DATE-BEG'])]

print(EITmetadata)

(91883,)
     DATE-BEG           DATE-AVG      XPOSURE CMDXPOS SHUTCLS FILTER WAVELNTH ... SC_X0 SC_Y0 SC_ROLL    HAEX_OBS       HAEY_OBS      HAEZ_OBS 
------------------ ------------------ ------- ------- ------- ------ -------- ... ----- ----- ------- -------------- -------------- -----------
2450098.5366403703  2450098.536677118    6.35    10.0     2.1  Al_+1      171 ...   0.0   0.0    0.01 -64908808000.0 130562576000.0 -77407740.0
 2450098.509462373 2450098.5129467477   602.1   600.0     2.1 Al_sup      284 ...   0.0   0.0    0.01 -64908808000.0 130562576000.0 -77407740.0
  2450099.07985044  2450099.079920463    12.1    10.0     2.1  Al_+1      171 ...   0.0   0.0   -0.04 -64908808000.0 130562576000.0 -77407740.0
2450098.5201983913  2450098.520678391  82.944    90.0     2.1 Al_sup      195 ...   0.0   0.0    0.01 -64908808000.0 130562576000.0 -77407740.0
 2450102.401938831 2450102.4025296876   102.1   100.0     2.1  Al_+1      304 ...   0.0   0.0   -0.01 -71817840000.0 1269393680

In [49]:
darkImgData =np.loadtxt("./EIT/darkImgFluxes_nobakeout.txt")
print(np.shape(darkImgData))

results = np.genfromtxt("./EIT/EITresults/EIT_LC.txt")
results_revisedBakeout = copy.deepcopy(results)

for y in range(2018,2023):
    results_yr = np.genfromtxt("./EIT/EITresults/EIT_LC_{0}_revisedBakeout.txt".format(y))
    first_row_yr = (results_revisedBakeout[:,0] == results_yr[0,0])
    first_idx_yr = np.arange(len(results_revisedBakeout))[first_row_yr][0]

    results_revisedBakeout[first_idx_yr:(first_idx_yr+len(results_yr))] = results_yr


pinholes = np.genfromtxt("./EIT/EITresults/EIT_pinholes.txt")
cameraErrors = np.genfromtxt("./EIT/EITresults/EIT_cameraErrors.txt")

results_revisedBakeout = results_revisedBakeout[np.argsort(results_revisedBakeout[:,0])]
pinholes = pinholes[np.argsort(pinholes[:,0])]
cameraErrors = cameraErrors[np.argsort(cameraErrors[:,0])]

(2467, 3)


In [58]:
print(np.shape(results_revisedBakeout))
print(np.shape(pinholes))
print(np.shape(cameraErrors))
print(np.shape(EITmetadata))

mismatch = (results_revisedBakeout[:,0] != pinholes[:,0])
print(np.shape(results_revisedBakeout[mismatch]))

mismatch = (results_revisedBakeout[:,0] != cameraErrors[:,0])
print(np.shape(results_revisedBakeout[mismatch]))

mismatch = ~np.isclose(results_revisedBakeout[:,0],EITmetadata['DATE-BEG'],rtol=0.01) 
print(np.shape(results_revisedBakeout[mismatch]))


(91883, 5)
(91883, 3)
(91883, 3)
(91883,)
(0, 5)
(0, 5)
(0, 5)


In [66]:
Al_filterMask = (EITmetadata['FILTER'] == 'Al_+1')
print(np.shape(EITmetadata[filterClearMask]))

#get rid of spurious detections of pinholes where there can't have been any because the filter was in place
pinholes[:,2][Al_filterMask] = 0

(85797,)


In [67]:
results_toSave = np.vstack((results_revisedBakeout.T, pinholes[:,2].T, cameraErrors[:,2].T)).T
print(np.shape(results_toSave))

np.savetxt("./EIT/EITresults/EIT_LC_withflags.txt",results_toSave,fmt="%f %f %d %f %f %d %d",header="t_obs[JD] solar_distance[m] wavelength[angstrom] unnorm_flux[DN/s] unnorm_flux_unc[DN/s] pinholes[1ifYes] cameraError[1ifYes]")


(91883, 7)
